In [ ]:
!pip install nltk rouge-score bert-score

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    T5Tokenizer,
    T5ForConditionalGeneration
)
from transformers.models.t5.modeling_t5 import T5LayerFF
from huggingface_hub import hf_hub_download
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
from rouge_score import rouge_scorer
import bert_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

!unzip -qn /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

base_bert_path = "emilyalsentzer/Bio_ClinicalBERT"
base_t5_path = "t5-small"
finetuned_bert_path = (
    "BioClinicalBert-MLM-Finetuned-40k-25epoch-exp-25epoch-questions.pth"
)
finetuned_t5_path = "T5-Finetuned-15k-20epoch.pth"
dataset_file_name = (
    "MEDMCQA-dataset-with-CLS-40k-25epoch-exp-25epoch-questions-nltk.json"
)
test_dataset_file_name = "MEDMCQA-test-dataset-4k.json"
repo_id = ""

test_batch_size = 32
bottleneck_size = 32
k = 1  # KNN hyperparameter

# Load Fine-tuned BERT Model from Hugging Face

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained(base_bert_path)
bert_model = BertForMaskedLM.from_pretrained(base_bert_path).to(device)

checkpoint_file = hf_hub_download(repo_id=repo_id, filename=finetuned_bert_path)
checkpoint = torch.load(checkpoint_file)
bert_model.load_state_dict(checkpoint["model_state_dict"])
bert_model = bert_model.bert  # Dropping MLM head
bert_model.eval()

# Load Pre-processed Dataset from Hugging Face

In [ ]:
dataset_path = hf_hub_download(repo_id=repo_id, filename=dataset_file_name)
loaded_df = pd.read_json(dataset_path)
display(loaded_df)

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)


# Vectorized Version
def encode_text(text, tokenizer, bert_model, max_length=128):
    if not isinstance(text, str):
        text = [preprocess_text(t) for t in text]
    else:
        text = preprocess_text(text)

    tokens = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        outputs = bert_model(**tokens)

    if "pooler_output" in outputs:
        cls_embedding = outputs.pooler_output
    elif "last_hidden_state" in outputs:
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    else:
        raise Exception("No CLS token found in the given model")

    return cls_embedding.cpu()

In [ ]:
# Vectorized Version
def cosine_similarity(queries, dataset):
    query_norms = queries / np.linalg.norm(queries, axis=1, keepdims=True)
    dataset_norms = dataset / np.linalg.norm(dataset, axis=1, keepdims=True)
    similarities = np.dot(query_norms, dataset_norms.T)
    return similarities


# Vectorized Version
def MSE_similarity(queries, dataset):
    dists = np.sum((queries[:, np.newaxis, :] - dataset) ** 2, axis=2)
    similarities = (
        1.0 / dists
    )  # inverse of distance scores are equivalent to similarity
    return similarities


# Vectorized Version
def k_nearest_embeddings(queries, dataset, k, similarity_metric=cosine_similarity):
    similarities = similarity_metric(queries, dataset)

    # Get the indices of the top k highest similarities for each query
    nearest_indices = np.argpartition(similarities, -k, axis=1)[:, -k:]

    # Sort these indices by the actual similarities
    sorted_indices = np.argsort(
        similarities[np.arange(similarities.shape[0])[:, None], nearest_indices], axis=1
    )[:, ::-1]

    # Gather the top k indices for each query
    top_k_indices = nearest_indices[
        np.arange(nearest_indices.shape[0])[:, None], sorted_indices
    ]

    # Get the top k similarities and corresponding embeddings for each query
    top_k_similarities = np.take_along_axis(similarities, top_k_indices, axis=1)
    top_k_embeddings = np.take(dataset, top_k_indices, axis=0)

    return top_k_indices, top_k_embeddings, top_k_similarities

# Sample Usage of Retrieval 

In [ ]:
queries = ["female with neck swelling. Gross and histology.", "another query"]
cls_embs = encode_text(queries, bert_tokenizer, bert_model).numpy().squeeze()

nearest_indices, _, nearest_similarities = k_nearest_embeddings(
    cls_embs,
    np.asarray(loaded_df["question_cls"].tolist()),
    k=2,
    similarity_metric=cosine_similarity,
)

for i, q in enumerate(queries):
    print(f"Input query: {q}")
    print("Cosine similarities of the k nearest embeddings:", nearest_similarities[i])
    mins_cosine = loaded_df.iloc[nearest_indices[i]]
    mins_cosine = mins_cosine.reset_index(drop=True)
    display(mins_cosine)
    print("\n\n")

# Load Fine-tuned T5 Model from Hugging Face

In [ ]:
# Adapter layer
class AdapterLayer(nn.Module):
    def __init__(self, emb_dim: int, bottleneck_size: int):

        super().__init__()

        self.sharif_llm_adapter = nn.Sequential(
            nn.Linear(emb_dim, bottleneck_size),
            nn.ReLU(),
            nn.Linear(bottleneck_size, emb_dim),
        )

    def forward(self, x: torch.Tensor):
        adapter_output = self.sharif_llm_adapter(x)
        output = x + adapter_output
        return output


class FeedForwardAdapterWrapper(nn.Module):
    def __init__(self, original_module: T5LayerFF, bottleneck_size: int):

        super().__init__()
        assert isinstance(original_module, T5LayerFF)

        self.original_module = original_module
        emb_dim = original_module.DenseReluDense.wi.in_features
        self.adapter = AdapterLayer(emb_dim, bottleneck_size)

    def forward(self, x: torch.Tensor):
        output = self.original_module(x)
        output = self.adapter(output)
        return output

In [ ]:
def mutate_model_recursive(model: nn.Module, bottleneck_size: int):
    for name, module in model.named_children():
        if isinstance(module, T5LayerFF):
            feed_forward_with_adapter = FeedForwardAdapterWrapper(
                module, bottleneck_size
            )
            setattr(model, name, feed_forward_with_adapter)
        else:
            mutate_model_recursive(module, bottleneck_size)


def mutate_model(model: nn.Module, bottleneck_size: int):
    if hasattr(model, "_mutated"):
        print("Model already contains adapter layers! \n Try reloading the model.")
        return

    mutate_model_recursive(model, bottleneck_size)

    model._mutated = True

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained(base_t5_path)
finetuned_t5_model = T5ForConditionalGeneration.from_pretrained(base_t5_path)

mutate_model(finetuned_t5_model, bottleneck_size=bottleneck_size)

checkpoint_file = hf_hub_download(repo_id=repo_id, filename=finetuned_t5_path)
finetuned_t5_model.load_state_dict(torch.load(checkpoint_file)["model_state_dict"])
finetuned_t5_model = finetuned_t5_model.to(device)

# Evaluation on Test Data

In [ ]:
opt_idx2str = {
    0: "A",
    1: "B",
    2: "C",
    3: "D",
}
opt_str2idx = {s: i for i, s in opt_idx2str.items()}


def dict_of_lists_to_list_of_dicts(dict_of_lists):
    keys = list(dict_of_lists.keys())

    list_of_dicts = []
    for i in range(len(dict_of_lists[keys[0]])):
        new_dict = {}
        for key in keys:
            new_dict[key] = dict_of_lists[key][i]
        list_of_dicts.append(new_dict)

    return list_of_dicts


def rag(
    query_list,
    retrieval_dataset,
    bert_model,
    bert_tokenizer,
    t5_model,
    t5_tokenizer,
    k=1,
    similarity_metric=cosine_similarity,
):
    question_list = [qeury["question"] for qeury in query_list]
    cls_embs = encode_text(question_list, bert_tokenizer, bert_model).numpy().squeeze()

    nearest_indices, _, _ = k_nearest_embeddings(
        cls_embs,
        np.asarray(retrieval_dataset["question_cls"].tolist()),
        k,
        similarity_metric=similarity_metric,
    )

    # Retrieve the explanations for the top k embeddings for each query
    for i, query in enumerate(query_list):
        mins_cosine = retrieval_dataset.iloc[nearest_indices[i]]
        mins_cosine = mins_cosine.reset_index(drop=True)
        exps = mins_cosine["exp"].tolist()

        exps_str = "\n".join(exps)
        query["exp"] = exps_str

    answer = generate_answer(query_list, t5_model, t5_tokenizer)

    return answer


# Vectorized Version
def generate_answer(questions, t5_model, t5_tokenizer):
    input_texts = [generate_prompt(q) for q in questions]
    inputs = t5_tokenizer(
        input_texts, return_tensors="pt", padding=True, truncation=True, max_length=512
    )
    inputs = inputs.to(device)

    t5_model.eval()
    with torch.no_grad():
        outputs = t5_model.generate(
            **inputs, max_length=512, num_beams=5, early_stopping=True
        )

    answers = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answers


# Used to generate prompts for the T5 model
def generate_prompt(q):
    prompt = f"Question: {q['question']}\n\nOptions:\nA: {q['opa']}\nB: {q['opb']}\nC: {q['opc']}\nD: {q['opd']}\n\nExplanation: {q['exp']}\n\nAnswer:"
    return prompt


# Only used to print the questions in the notebook
def generate_question(q):
    question_str = (
        f"{q['question']}\nA) {q['opa']}\nB) {q['opb']}\nC) {q['opc']}\nD) {q['opd']}"
    )
    return question_str

In [ ]:
def convert_answer(answer):
    prefix_str = "Answer: "
    if answer.startswith(prefix_str):
        try:
            option = answer[len(prefix_str) :]
            return opt_str2idx[option]
        except:
            return 100
    return 100


def show_classification_metrics(trues, preds):
    preds = [convert_answer(answer) for answer in preds]
    trues = [convert_answer(answer) for answer in trues]

    accuracy = accuracy_score(trues, preds)
    macro_f1 = f1_score(trues, preds, average="macro")
    micro_f1 = f1_score(trues, preds, average="micro")
    macro_precision = precision_score(trues, preds, average="macro", zero_division=0)
    macro_recall = recall_score(trues, preds, average="macro", zero_division=0)
    conf_matrix = confusion_matrix(trues, preds)

    print(f"Accuracy        =  {accuracy * 100:.2f}%")
    print(f"Macro F1-score  =  {macro_f1 * 100:.2f}%")
    print(f"Micro F1-score  =  {micro_f1 * 100:.2f}%")
    print(f"Macro Precision =  {macro_precision * 100:.2f}%")
    print(f"Macro Recall    =  {macro_recall * 100:.2f}%")

    if (
        100 in preds
    ):  # Model's answer for some questions is not among the question options
        class_names = ["Option A", "Option B", "Option C", "Option D", "None"]
    else:  # Model's answer for every single question is among the provided options
        class_names = ["Option A", "Option B", "Option C", "Option D"]

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        conf_matrix,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=class_names,
        yticklabels=class_names,
    )
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
# BLEU Scores
def calculate_bleu_scores(generated, references):
    smoothing = SmoothingFunction().method2
    bleu1 = []
    bleu2 = []
    bleu3 = []
    bleu4 = []

    for gen, ref in zip(generated, references):
        ref_list = [ref.split()]
        gen_list = gen.split()
        bleu1.append(
            sentence_bleu(
                ref_list, gen_list, weights=(1, 0, 0, 0), smoothing_function=smoothing
            )
        )
        bleu2.append(
            sentence_bleu(
                ref_list,
                gen_list,
                weights=(0.5, 0.5, 0, 0),
                smoothing_function=smoothing,
            )
        )
        bleu3.append(
            sentence_bleu(
                ref_list,
                gen_list,
                weights=(0.33, 0.33, 0.33, 0),
                smoothing_function=smoothing,
            )
        )
        bleu4.append(
            sentence_bleu(
                ref_list,
                gen_list,
                weights=(0.25, 0.25, 0.25, 0.25),
                smoothing_function=smoothing,
            )
        )

    bleu1 = sum(bleu1) / len(bleu1)
    bleu2 = sum(bleu2) / len(bleu2)
    bleu3 = sum(bleu3) / len(bleu3)
    bleu4 = sum(bleu4) / len(bleu4)

    return bleu1, bleu2, bleu3, bleu4


# ROUGE Scores
def calculate_rouge_scores(generated, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge1 = []
    rouge2 = []
    rougel = []

    for gen, ref in zip(generated, references):
        scores = scorer.score(ref, gen)
        rouge1.append(scores["rouge1"].fmeasure)
        rouge2.append(scores["rouge2"].fmeasure)
        rougel.append(scores["rougeL"].fmeasure)

    rouge1 = sum(rouge1) / len(rouge1)
    rouge2 = sum(rouge2) / len(rouge2)
    rougel = sum(rougel) / len(rougel)

    return rouge1, rouge2, rougel


# BERT Scores
def calculate_bert_scores(generated, references):
    P, R, F1 = bert_score.score(
        generated, references, lang="en", rescale_with_baseline=True
    )
    return P.mean().item(), R.mean().item(), F1.mean().item()


def show_text_generation_metrics(trues, preds):
    bleu1, bleu2, bleu3, bleu4 = calculate_bleu_scores(preds, trues)
    rouge1, rouge2, rougel = calculate_rouge_scores(preds, trues)
    bert_p, bert_r, bert_f1 = calculate_bert_scores(preds, trues)
    print("\n")

    print(f"BLEU-1         =  {bleu1:.5f}")
    print(f"BLEU-2         =  {bleu2:.5f}")
    print(f"BLEU-3         =  {bleu3:.5f}")
    print(f"BLEU-4         =  {bleu4:.5f}", end="\n\n")

    print(f"ROUGE-1        =  {rouge1:.5f}")
    print(f"ROUGE-2        =  {rouge2:.5f}")
    print(f"ROUGE-L        =  {rougel:.5f}", end="\n\n")

    print(f"BERT Precision =  {bert_p:.5f}")
    print(f"BERT Recall    =  {bert_r:.5f}")
    print(f"BERT F1 Score  =  {bert_f1:.5f}")

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self._df = df.fillna("")

    def __len__(self):
        return len(self._df)

    def __getitem__(self, idx):
        return dict(self._df.iloc[idx])


test_dataset_path = hf_hub_download(repo_id=repo_id, filename=test_dataset_file_name)
loaded_test_df = pd.read_json(test_dataset_path)

test_dataset = TestDataset(loaded_test_df)
test_loader = DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4
)

# Evaluation on Test Data with Finetuned T5

In [ ]:
for k in [1, 2, 3]:
    print(f"========================= Evaluating with k={k} =========================")

    preds = []
    trues = []
    for test_queries in tqdm(test_loader, leave=False):
        model_answers = rag(
            dict_of_lists_to_list_of_dicts(test_queries),
            loaded_df,
            bert_model,
            bert_tokenizer,
            finetuned_t5_model,
            t5_tokenizer,
            k=k,
        )
        preds += model_answers
        trues += [f"Answer: {opt_idx2str[cop]}" for cop in test_queries["cop"].tolist()]

    show_classification_metrics(trues, preds)
    show_text_generation_metrics(trues, preds)

# Evaluation on Test Data using T5 WITHOUT Finetuning

In [ ]:
t5_model = T5ForConditionalGeneration.from_pretrained(base_t5_path).to(device)

for k in [1, 2, 3]:
    print(f"========================= Evaluating with k={k} =========================")

    preds = []
    trues = []
    for test_queries in tqdm(test_loader, leave=False):
        model_answers = rag(
            dict_of_lists_to_list_of_dicts(test_queries),
            loaded_df,
            bert_model,
            bert_tokenizer,
            t5_model,
            t5_tokenizer,
            k=k,
        )
        preds += model_answers
        trues += [f"Answer: {opt_idx2str[cop]}" for cop in test_queries["cop"].tolist()]

    show_classification_metrics(trues, preds)
    show_text_generation_metrics(trues, preds)

# Evaluation on Custom Samples

The following samples are generated based on the information in the original MedMCQA dataset. These samples are NOT taken directly from the dataset.

In [ ]:
eval_queries = [
    {
        "question": "What is the term for the active searching for unrecognized problems in apparently healthy individuals using quick tests?",
        "opa": "Vaccination",
        "opb": "Screening",
        "opc": "Monitoring",
        "opd": "Diagnosis",
        "cop": 1,
    },
    {
        "question": "Which of the following causes a decrease in ESR?",
        "opa": "Sickle cell anaemia",
        "opb": "Inflammation",
        "opc": "COVID-19",
        "opd": "Pregnancy",
        "cop": 0,
    },
    {
        "question": "Which option could be identified using the cephalic index?",
        "opa": "Blood type",
        "opb": "Sex",
        "opc": "Hair color",
        "opd": "Race",
        "cop": 3,
    },
    {
        "question": "In which case is a magistrate inquest NOT required?",
        "opa": "Death in police custody",
        "opb": "Death in police firing",
        "opc": "Death by suicide",
        "opd": "Death in psychiatry hospital",
        "cop": 2,
    },
    {
        "question": "Which test is most related to Addison's disease (i.e. adrenal insufficiency)?",
        "opa": "ACTH (Cosyntropin) test",
        "opb": "Blood glucose test",
        "opc": "MRI of the adrenal glands",
        "opd": "CT scan of abdomen",
        "cop": 0,
    },
    {
        "question": "For a woman in her 30s, under which circumstance, there is an increased risk of having a baby with Down syndrome?",
        "opa": "Undergoing IVF treatment",
        "opb": "Having a previous baby with Klinefelter syndrome",
        "opc": "Having three first-trimester miscarriages",
        "opd": "Having a previous baby with Turner syndrome",
        "cop": 2,
    },
    {
        "question": "Which one is inhibited by Azaserine?",
        "opa": "Ribose-phosphate diphosphokinase",
        "opb": "Dihydrofolate reductase",
        "opc": "Glycinamide ribonucleotide transformylase",
        "opd": "Formyl glycinamide ribonucleotide amidotransferase (aka PurL)",
        "cop": 3,
    },
    {
        "question": "Which metric provides information about the completed family size?",
        "opa": "Pregnancy rate",
        "opb": "General marital fertility rate",
        "opc": "Gross reproductive rate",
        "opd": "Total fertility rate",
        "cop": 3,
    },
    {
        "question": "What is the most frequent complication associated with a Colles fracture?",
        "opa": "Stiffness of fingers",
        "opb": "Compartment syndrome",
        "opc": "Malunion",
        "opd": "Carpal tunnel syndrome",
        "cop": 0,
    },
    {
        "question": "How does radiotherapy work?",
        "opa": "By using ultrasound tissue scanning",
        "opb": "By ionization of tissues",
        "opc": "By blocking hormone receptors",
        "opd": "By necrosis of body cells",
        "cop": 1,
    },
    {
        "question": "The ligament of Berry in the thyroid gland attaches it to which structure?",
        "opa": "Larynx",
        "opb": "Cricoid cartilage",
        "opc": "Esophagus",
        "opd": "Thyroid",
        "cop": 1,
    },
    {
        "question": "Conn syndrome is caused by overproduction of which hormone?",
        "opa": "ACTH",
        "opb": "ADH",
        "opc": "Aldosterone",
        "opd": "Cortisol",
        "cop": 2,
    },
]

## Evaluation using T5 with Fine-tuning

Note that the template used here to print the questions, is in a DIFFERENT format than the prompt given to the T5 model.

In [ ]:
preds = []
trues = []
model_answers = rag(
    eval_queries,
    loaded_df,
    bert_model,
    bert_tokenizer,
    finetuned_t5_model,
    t5_tokenizer,
    k=k,
)
for i, (question, model_ans) in enumerate(zip(eval_queries, model_answers)):
    print(
        f"========================= QUESTION {i+1:>2} / {len(eval_queries)} ========================="
    )

    question_str = generate_question(question)
    print(question_str, end="\n\n")

    print(f'Model\'s output =  "{model_ans}"')

    correct_ans = f"Answer: {opt_idx2str[question['cop']]}"
    print(f'Correct output =  "{correct_ans}"', end="\n\n")

    preds.append(model_ans)
    trues.append(correct_ans)

In [ ]:
show_classification_metrics(trues, preds)
show_text_generation_metrics(trues, preds)

## Evaluation using T5 WITHOUT Fine-tuning

In [ ]:
preds = []
trues = []
model_answers = rag(
    eval_queries,
    loaded_df,
    bert_model,
    bert_tokenizer,
    t5_model,
    t5_tokenizer,
    k=k,
)
for i, (question, model_ans) in enumerate(zip(eval_queries, model_answers)):
    print(
        f"========================= QUESTION {i+1:>2} / {len(eval_queries)} ========================="
    )

    question_str = generate_question(question)
    print(question_str, end="\n\n")

    print(f'Model\'s output =  "{model_ans}"')

    correct_ans = f"Answer: {opt_idx2str[question['cop']]}"
    print(f'Correct output =  "{correct_ans}"', end="\n\n")

    preds.append(model_ans)
    trues.append(correct_ans)

In [ ]:
show_classification_metrics(trues, preds)
show_text_generation_metrics(trues, preds)